In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time 
import re
import pandas as pd
import mysql.connector
import pymysql

In [2]:
url ='https://www.kurashiru.com/recipes/459512b1-1c95-4a59-917d-4363f440d0a7'
def get_html(url):
    time.sleep(2)
    res = requests.get(url)
    res.encoding = 'utf8' 
    soup = BeautifulSoup(res.text,'html.parser')
    return soup

def get_page(url):
    result = {}
    soup = get_html(url)
    
    json_elem = soup.find(id="hydration_initial_state")
    json_str = get_text(json_elem)
    json_load = json.loads(json_str)
    data_dic = json_load['state']['fetchVideo']['data']['data']
    
    recipe_id = data_dic['id']
    recipe_name = data_dic["attributes"]["title"]
    ingredients = data_dic["attributes"]["ingredients"]
    result["recipe_id"] = recipe_id
    result["recipe_name"] = recipe_name
    
    ingredient_list = []
    for ingredient in ingredients:
        ingredient_dic = {}
        
        ingredient_id = ingredient["id"]
        ingredient_type = ingredient["type"]
                     
        if ingredient_type=="ingredients":
            ingredient_group_id = ingredient["group-id"]
            ingredient_title = ingredient["name"]
            ingredient_name = ingredient["actual-name"]
            ingredient_amount = ingredient["quantity-amount"]
                         
        else:
            ingredient_title = ingredient["title"]
            ingredient_group_id = 0
            ingredient_name = ""
            ingredient_amount = ""
        
        ingredient_dic["ingredient_name"] = ingredient_name
        ingredient_dic["ingredient_amount"] = ingredient_amount
                     
        ingredient_list.append(ingredient_dic)
        result["ingredient_list"] = ingredient_list
    return result
    

def get_text(elem):
    text = elem.string
    text = text.strip()  
    return text

result = get_page(url)
print(result)
ingredint_recipe =result['ingredient_list']
recipe = pd.DataFrame(ingredint_recipe)
recipe

{'recipe_id': '459512b1-1c95-4a59-917d-4363f440d0a7', 'recipe_name': '照り焼きチキン', 'ingredient_list': [{'ingredient_name': '鶏もも肉', 'ingredient_amount': '300g'}, {'ingredient_name': '片栗粉', 'ingredient_amount': '大さじ2'}, {'ingredient_name': '料理酒', 'ingredient_amount': '大さじ1'}, {'ingredient_name': 'しょうゆ', 'ingredient_amount': '大さじ1'}, {'ingredient_name': 'みりん', 'ingredient_amount': '大さじ1'}, {'ingredient_name': '砂糖', 'ingredient_amount': '小さじ1'}, {'ingredient_name': 'サラダ油', 'ingredient_amount': '小さじ2'}, {'ingredient_name': 'ベビーリーフ', 'ingredient_amount': '5g'}]}


,ingredient_name,ingredient_amount
0,鶏もも肉,300g
1,片栗粉,大さじ2
2,料理酒,大さじ1
3,しょうゆ,大さじ1
4,みりん,大さじ1
5,砂糖,小さじ1
6,サラダ油,小さじ2
7,ベビーリーフ,5g


In [4]:
#食品成分DBから参照
recipe_nu=recipe.assign(protein =[51.9, 0, 0.1,1.4,0.1, 0, 0,0.1  ],
                                            fat =[57.3, 0, 0, 0, 0, 0, 0, 0],
                                            carbs =[0,14.7, 0.7, 1.4, 7.8, 3, 8, 0.2],
                                            calorie = [702,61, 10,14,43,12, 71, 1])
recipe_nu

,ingredient_name,ingredient_amount,protein,fat,carbs,calorie
0,鶏もも肉,300g,51.9,57.3,0.0,702
1,片栗粉,大さじ2,0.0,0.0,14.7,61
2,料理酒,大さじ1,0.1,0.0,0.7,10
3,しょうゆ,大さじ1,1.4,0.0,1.4,14
4,みりん,大さじ1,0.1,0.0,7.8,43
5,砂糖,小さじ1,0.0,0.0,3.0,12
6,サラダ油,小さじ2,0.0,0.0,8.0,71
7,ベビーリーフ,5g,0.1,0.0,0.2,1


In [5]:
#２人前のため一人分にする
print(round(recipe_nu['protein'].sum()/2)),
print(round(recipe_nu['fat'].sum()/2)),
print(round(recipe_nu['carbs'].sum()/2)),
print(round(recipe_nu['calorie'].sum()/2))

27.0
29.0
18.0
457.0


In [67]:
cnx = None

try:
    cnx = mysql.connector.connect(
        user='root',  # ユーザー名
        password='',  # パスワード
        host='localhost',  # ホスト名(IPアドレス）
        database='recipe'  # データベース名
    )

    cursor = cnx.cursor()

    sql = ('''
    INSERT INTO recipe 
        (recipe_id, recipe_name, preference, category, protein, fat, carbs, calorie)
    VALUES 
         (%s, %s, %s, %s, %s, %s, %s, %s)
    ''')

    data = [('1', '照り焼きチキン', '5', '1','27','29', '18', '457')]

    cursor.executemany(sql, data)
    cnx.commit()

    print(f"{cursor.rowcount} records inserted.")

    cursor.close()

except Exception as e:
    print(f"Error Occurred: {e}")

finally:
    if cnx is not None and cnx.is_connected():
        cnx.close()

1 records inserted.
